In [72]:
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk,Image
import imutils
import cv2
import numpy as np

def stitch_pair(origin, addition):
    padtop = addition.shape[1]
    padbottom = 0
    padleft = 0
    padright = 0
    origin = cv2.copyMakeBorder(origin, padtop, padbottom, padleft, padright, cv2.BORDER_CONSTANT, (0,0,0))

    sift = cv2.ORB_create()
    _, mask = cv2.threshold(cv2.cvtColor(origin, cv2.COLOR_BGR2GRAY), 10, 255, cv2.THRESH_BINARY)
    kp_o, desc_o = sift.detectAndCompute(origin, mask)
    
    _, mask = cv2.threshold(cv2.cvtColor(addition, cv2.COLOR_BGR2GRAY), 10, 255, cv2.THRESH_BINARY)
    kp_a, desc_a = sift.detectAndCompute(addition, mask)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(desc_o, desc_a)
    # If no matches were found at all
    if len(matches)==0: 
        return False, origin
    
    matches = sorted(matches, key = lambda x:x.distance)
    
    out = cv2.drawMatches(origin,kp_o,addition,kp_a,matches[:len(matches)],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imshow("matches", out)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    src_points = np.float32([kp_a[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
    dst_points = np.float32([kp_o[m.queryIdx].pt for m in matches]).reshape(-1,1,2)

    ### add distance enforcing
    
    ###
    
    h, _ = cv2.estimateAffinePartial2D(src_points, dst_points)
    if checkAffine(addition, h):
        flag = True
        warp = cv2.warpAffine(addition, h, (origin.shape[1],origin.shape[0]))
        _, mask = cv2.threshold(cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY_INV)
    
        output = cv2.bitwise_and(origin, origin, mask=mask)
        output = output + warp
    else:
        flag = False
        print("using origin")
        output = origin.copy()
    
    _, mask = cv2.threshold(cv2.cvtColor(output, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[0])
    
    return flag, output[y:y+h, x:x+w]

def checkAffine(image, matrix):
    print(matrix)
    # А11 / А22 близко к единице - perserve scaling
    # A11 > 0, A22 > 0 - no reflecting
    # нужны ограничения для A12 и A21 - restrict shearing - спросить АВ
    
    # Детерминант матрицы преобразования. Если он < 0 то преобразование точно не сохраняет ориентацию,
    # Но пусть доп. условие что детерминант обязан быть больше 0.5
    determinant = matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    print("determinant: ", determinant)
    
    if np.fabs(determinant - 1) > 0.5: return False
    else: return True

def stitch_pair_legacy(origin, addition):
    padtop = addition.shape[1]
    padbottom = 0
    padleft = 0
    padright = 0
    origin = cv2.copyMakeBorder(origin, padtop, padbottom, padleft, padright, cv2.BORDER_CONSTANT, (0,0,0))

    sift = cv2.SIFT_create()
    _, mask = cv2.threshold(cv2.cvtColor(origin, cv2.COLOR_BGR2GRAY), 10, 255, cv2.THRESH_BINARY)
    kp_o, desc_o = sift.detectAndCompute(origin, mask)
    
    _, mask = cv2.threshold(cv2.cvtColor(addition, cv2.COLOR_BGR2GRAY), 10, 255, cv2.THRESH_BINARY)
    kp_a, desc_a = sift.detectAndCompute(addition, mask)

    bf = cv2.BFMatcher()
    matches = bf.match(desc_o, desc_a)
    # If no matches were found at all
    if len(matches)==0: 
        return False, origin
    
    matches = sorted(matches, key = lambda x:x.distance)
    
    out = cv2.drawMatches(origin,kp_o,addition,kp_a,matches[:len(matches)],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv2.imshow("matches", out)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    src_points = np.float32([kp_a[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
    dst_points = np.float32([kp_o[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    
    h, status = cv2.findHomography(src_points, dst_points, cv2.RANSAC)
    if (checkPerspective(addition, h)):
        flag = True
        warp = cv2.warpPerspective(addition, h, (origin.shape[1],origin.shape[0]))
        _, mask = cv2.threshold(cv2.cvtColor(warp, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY_INV)
    
        output = cv2.bitwise_and(origin, origin, mask=mask)
        output = output + warp
    else:
        print("Using origin")
        output = origin.copy()
        flag = False

    _, mask = cv2.threshold(cv2.cvtColor(output, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    x,y,w,h = cv2.boundingRect(contours[0])
    
    return flag, output[y:y+h, x:x+w]    
    
def checkPerspective(image, matrix):
    print(matrix)
    # Первая проверка -- отношение левой и правой стороны после трансформации 
    points = []
    new_points = []
    nw = (0,0)
    points.append(nw)
    ne = (image.shape[1],0)
    points.append(ne)
    se = (image.shape[1], image.shape[0])
    points.append(se)
    sw = (0, image.shape[0])
    points.append(sw)
    
    left_original = np.sqrt((nw[0] - sw[0])**2 + (nw[1] - sw[1])**2)
    right_original = np.sqrt((ne[0] - se[0])**2 + (ne[1] - se[1])**2)
    top_original = np.sqrt((nw[0] - ne[0])**2 + (nw[1]-ne[1])**2)
    bottom_original = np.sqrt((sw[0] - se[0])**2 + (sw[1] - se[1])**2)
    
    for p in points:
        alpha = matrix[2][0] * p[0] + matrix[2][1] * p[1] + matrix[2][2]
        y = (matrix[1][0] * p[0] + matrix[1][1] * p[1] + matrix[1][2] ) / alpha
        x = (matrix[0][0] * p[0] + matrix[0][1] * p[1] + matrix[0][2] ) / alpha
        new_points.append((x,y))
    new_nw = new_points[0]
    new_ne = new_points[1]
    new_se = new_points[2]
    new_sw = new_points[3]
    
    ### distance that every point travelled
    print(np.sqrt((new_nw[0] - nw[0])**2 + (new_nw[1] - nw[1])**2))
    print(np.sqrt((new_ne[0] - ne[0])**2 + (new_ne[1] - ne[1])**2))
    print(np.sqrt((new_se[0] - se[0])**2 + (new_se[1] - se[1])**2))
    print(np.sqrt((new_sw[0] - sw[0])**2 + (new_sw[1] - sw[1])**2))
    
    left_converted = np.sqrt((new_nw[0] - new_sw[0])**2 + (new_nw[1] - new_sw[1])**2)
    right_converted = np.sqrt((new_ne[0] - new_se[0])**2 + (new_ne[1] - new_se[1])**2)
    top_converted = np.sqrt((new_nw[0] - new_ne[0])**2 + (new_nw[1] - new_ne[1])**2)
    bottom_converted = np.sqrt((new_sw[0] - new_se[0])**2 + (new_sw[1] - new_se[1])**2)
    
    # Если модуль (отношение - 1) меньше чем 0.2, то вторая проверка:
    print("left right ratio: ", left_converted / right_converted)
    print("top bottom ratio: ", top_converted / bottom_converted)
    
    # Число обусловленности для плохих преобразований очень большое
    print("condition: ", np.linalg.cond(matrix[:,:2]))
    if np.linalg.cond(matrix[:, :2]) >= 10: return False
    if np.fabs(left_converted/right_converted - 1) >= 0.3: return False
    if np.fabs(top_converted/bottom_converted - 1) >= 0.3: return False
    
    # Детерминант матрицы преобразования. Если он < 0 то преобразование точно не сохраняет ориентацию,
    # Но пусть доп. условие что детерминант обязан быть больше 0.5
    determinant = matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
    print("determinant: ", determinant)
    
    if (determinant < 0): return False
    else: return True

In [22]:
paths = []
paths = filedialog.askopenfilenames(initialdir="/home/raz/src/indor/imagestitcher/Images/", title="Select file",filetypes=(("PNG",".png"),("JPEG",".jpg .jpeg")))
print(len(paths))

N_IMAGES = len(paths)
images = []
for p in paths:
    images.append(cv2.imread(p))

36


In [73]:
process = images[0].copy()
helper = []
d = 2.75
for i in range(1,N_IMAGES):
    print(i)
    temp = process.copy()
    status, process = stitch_pair(process, images[i])
    d += 2.75
    if not(status):
        d = 2.75
        helper.append(temp.copy())
        cv2.imwrite("/home/raz/src/indor/imagestitcher/Images/debug/"+str(i)+".png", temp)
        _, process = stitch_pair(images[i-1], images[i])

process = helper[0]
for i in range(len(helper)):
    _, process = stitch_pair(process, helper[i])

cv2.imwrite("/home/raz/src/indor/imagestitcher/Images/debug/Result.png", process)
cv2.imshow("Result", process)
cv2.waitKey(0)    
cv2.destroyAllWindows()

1
[[ 9.80718822e-01  2.10122258e-02 -2.52061218e+00]
 [-2.10122258e-02  9.80718822e-01  6.49402078e+02]]
determinant:  0.9622509221048117
2
[[ 9.90163677e-01  4.04697507e-02 -2.00420767e+01]
 [-4.04697507e-02  9.90163677e-01  6.45643619e+02]]
determinant:  0.9820619088413646
3
[[ 9.82249237e-01  3.91446705e-02 -1.98024686e+01]
 [-3.91446705e-02  9.82249237e-01  6.46636813e+02]]
determinant:  0.9663458689638308
4
[[ 9.81487368e-01  4.47781999e-02 -2.42432693e+01]
 [-4.47781999e-02  9.81487368e-01  6.47340315e+02]]
determinant:  0.9653225416938604
5
[[ 9.79239801e-01  4.90870613e-02 -2.97649018e+01]
 [-4.90870613e-02  9.79239801e-01  6.47381698e+02]]
determinant:  0.9613201269092083
6
[[ 9.66695505e-01  5.58868865e-02 -2.99198262e+01]
 [-5.58868865e-02  9.66695505e-01  6.71292678e+02]]
determinant:  0.937623543555056
7
[[ 9.31201683e-01  7.97720461e-02 -2.91463575e+01]
 [-7.97720461e-02  9.31201683e-01  6.92540792e+02]]
determinant:  0.8735001537508529
8
[[ 9.05511663e-01  9.62039432e-02